In [12]:
import os
import pathlib
pcap_fnames = [str(pathlib.Path(root) / file) for root, subdirs, files in os.walk('./icmp') for file in files if file.endswith('.pcap')]
pcap_fnames.sort()
print(pcap_fnames[:5])

def fname_to_day_and_honeypot_ip(fname):  # TODO move to utils
    # 'icmp/ICMP_2019-09-05_to_172.31.21.94.pcap' -> ['2019-09-05', '172.31.21.94']
    return fname.replace('.pcap', '').split('ICMP_')[1].split('_to_')

['icmp/ICMP_2019-09-05_to_172.31.21.94.pcap', 'icmp/ICMP_2019-09-06_to_172.31.21.94.pcap', 'icmp/ICMP_2019-09-07_to_172.31.21.94.pcap', 'icmp/ICMP_2019-09-08_to_172.31.21.94.pcap', 'icmp/ICMP_2019-09-09_to_172.31.21.94.pcap']


In [13]:
from pypacker import ppcap
from pypacker.layer12 import ethernet
from pypacker.layer3 import ip
from pypacker.layer4 import tcp
from pypacker.layer4 import udp
from pypacker.layer3 import icmp
from collections import Counter

In [14]:
pwriter = ppcap.Writer(filename='icmp_all.pcap', linktype=ppcap.DLT_EN10MB)

for fname in pcap_fnames:
    day, honeypot_ip = fname_to_day_and_honeypot_ip(fname)
    preader = ppcap.Reader(filename=fname)
    for ts, buf in preader:
        eth = ethernet.Ethernet(buf)
        if eth[ethernet.Ethernet, ip.IP] is not None:
            #if eth[ip.IP].src_s == honeypot_ip:
            #    continue  # skip packets emitted by honeypot
            pwriter.write(eth.bin())                    

pwriter.close()